<a href="https://colab.research.google.com/github/Kane0068/Project-for-Data-Science/blob/main/Anahtar_%C5%9Eehir_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Önce görüntü mü yoksa pdf mi ona karar verir
- Görüntüyse ona göre
* pdf ise ona göre okuma işlemi yapar
- İçinde tanımlanan şehir isimlerini tarar
- Bulursa uygun kategoriye ekler bulamazsa boş döndürür


In [ ]:
import cv2
import pytesseract
import PyPDF2
import re

def categorize_file(input_file, cities, category_folder):
    # PDF mi görüntü dosyası mı kontrol ediyoruz
    if input_file.lower().endswith(('.pdf', '.jpeg', '.jpg', '.png')):
        is_image = True
    else:
        is_image = False

    # Görüntü ise
    if is_image:
        image = cv2.imread(input_file)
        text = pytesseract.image_to_string(image)
    # Metin formatı ise
    else:
        pdf_file = open(input_file, "rb")
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        text = ""
        for page_num in range(pdf_reader.numPages):
            page = pdf_reader.getPage(page_num)
            text += page.extractText()
        pdf_file.close()

    # Şehir listesini yapalım
    matched_cities = [city for city in cities if re.search(city, text, re.I)]

    if matched_cities:
        # Kategoriyi belirle
        category = matched_cities[0]

        # Kategorinin dosya yolu
        category_path = os.path.join(category_folder, category)

        # Kategorinin klasörünü oluştur
        os.makedirs(category_path, exist_ok=True)

        # Metni ilgili kategoriye kaydet
        with open(os.path.join(category_path, os.path.basename(input_file)), "w") as file:
            file.write(text)

    return matched_cities

# Kullanım örneği:
input_file = "example.pdf"  # İşlem yapılacak dosyanın yolu
cities = ["Adana", "İstanbul", "Ankara"]  # İlgili şehirleri ekleyelim
category_folder = "categorized_data"  # Kategorilendirilmiş verilerin kaydedileceği klasör

matched_cities = categorize_file(input_file, cities, category_folder)
if matched_cities:
    print(f"Kategori: {matched_cities[0]}")


- TARAYICI KULLANARAK TARA
- TARANAN GÖRÜNTÜYÜ METNE ÇEVİR
- İŞLEMLERİ TEKRARLA

In [ ]:
import cv2
import pytesseract
import PyPDF2
import re
import shutil
import os
from scanner import Scanner  # Tarayıcı kütüphanesi

def scan_and_categorize(input_path, cities, category_folder):
    scanner = Scanner()  # Tarayıcı nesnesi oluşturun
    scanned_image = scanner.scan_to_file()  # Tarayıcıyı kullanarak bir görüntü alın
    input_file = input_path

    if not input_file.lower().endswith(('.pdf', '.jpeg', '.jpg', '.png')):
        raise ValueError("Desteklenmeyen dosya uzantısı.")

    # Görüntüyü metne çevir
    image = cv2.imread(scanned_image)
    text = pytesseract.image_to_string(image)

    # Şehir listesi
    matched_cities = [city for city in cities if re.search(city, text, re.I)]

    if matched_cities:
        # Kategoriyi belirle
        category = matched_cities[0]

        # Kategorinin dosya yolu
        category_path = os.path.join(category_folder, category)

        # Kategorinin klasörünü oluştur
        os.makedirs(category_path, exist_ok=True)

        # Metni ilgili kategoriye kaydet
        with open(os.path.join(category_path, os.path.basename(input_file)), "w") as file:
            file.write(text)

        # Tarayıcı tarafından kaydedilen geçici görüntüyü sil
        os.remove(scanned_image)

    return matched_cities

# Kullanım örneği:
input_path = "example.jpg"  # Tarayıcı ile tarama yapılacak dosyanın yolu
cities = ["Adana", "İstanbul", "Ankara"]  # İlgili şehirleri ekleyin
category_folder = "categorized_data"  # Kategorilendirilmiş verilerin kaydedileceği klasör

matched_cities = scan_and_categorize(input_path, cities, category_folder)
if matched_cities:
    print(f"Kategori: {matched_cities[0]}")
